##**Atividade Avaliativa - Processamento de Linguagem Natural**
### **Equipe:** Ana Clara, Bruno, Igor e Pedro Cruz


# **Descrição e motivação do problema:**

A disseminação de **fake news** tem acontecido com cada vez mais frequência e possui impacto significativo na sociedade, contribuindo para a desinformação, polarização social e decisões equivocadas. A motivação para desenvolver um algoritmo de identificação de notícias falsas está em promover uma sociedade mais informada, fornecendo as pessoas uma ferramenta automatizada para avaliar a veracidade de informações rapidamente. Isso pode ajudar a mitigar os efeitos negativos da desinformação, capacitando as pessoas a tomarem decisões mais conscientes e baseadas em fatos confiáveis.

# **Descrição da base de dados:**

A base de dados que decidimos utilizar é a [Fake.Br Corpus](https://github.com/roneysco/Fake.br-Corpus). Ela fornece pares de notícias verdadeiras e falsas em português.

Mais especificamente, iremos utilizar os dados contidos na pasta "size_normalized_texts", que contém versões truncadas dos textos, onde em cada par verdadeiro-falso o texto mais longo é truncado (em número de palavras) para o tamanho do texto mais curto. Esta versão do corpus pode ser útil para evitar inclinações incorretas em experimentos de aprendizado de máquina.

# **Objetivo de negócio ou científico associado ao problema:**


O objetivo científico deste projeto é desenvolver e validar um algoritmo capaz de identificar fake news baseado no que ele sabe. Já o objetivo de negócio é fornecer uma solução automatizada, que permita organizações, jornalistas e usuários individuais verificarem rapidamente a veracidade de informações, reduzindo os impactos da desinformação em escala e promovendo a confiança em fontes confiáveis.

# **Pré-processamento & Extração de Características:**

Fazendo todas as importações necessárias para o notebook

In [17]:
import zipfile
import os
import pandas as pd
import torch
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline

Unzipando os arquivos baixados

In [18]:
caminho_zip = 'Fake.br-Corpus-master.zip'

with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
    zip_ref.extractall('arquivos')

Obtendos os dados da pasta size_normalized_texts, como comentado anteriormente, e colocando tudo em um dataframe só, inserindo o label 0 ou 1 para distinguirmos entre verdadeiro ou falso.

In [19]:
fake_dir = 'arquivos/Fake.br-Corpus-master/size_normalized_texts/fake'
true_dir = 'arquivos/Fake.br-Corpus-master/size_normalized_texts/true'

def load_texts(directory, label):
    data = []
    for file in os.listdir(directory):
        file_path = os.path.join(directory, file)
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                text = f.read().strip()
                if text:
                    data.append({'text': text, 'label': label})
        except Exception as e:
            print(f"Erro ao ler o arquivo {file_path}: {e}")
    return data

fake_data = load_texts(fake_dir, 0)  # 0 para "falso"
true_data = load_texts(true_dir, 1)  # 1 para "verdadeiro"

df = pd.DataFrame(fake_data + true_data)

print(df.head())

                                                text  label
0  Acorda Brasil! O texto que pode anistiar os cr...      0
1  Beto Barbosa é liberado e pede desculpas à PM:...      0
2  O "ogro" do senado: Destemperado,  Lindbergh d...      0
3  Serginho – Esse é o verdadeiro herói do Brasil...      0
4  BBB 16 mal começou e já causa polêmicas. Supos...      0


Analisando tamanho

In [20]:
df.shape

(7200, 2)

Verificando se temos algum nulo

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7200 entries, 0 to 7199
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    7200 non-null   object
 1   label   7200 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 112.6+ KB


Baixando a library dos transformers

In [22]:
pip install transformers

Inicializando o tokenizador para transformar textos em tokens numéricos e carregando o modelo pré-treinado BERT para extração de características.

In [23]:
PRE_TRAINED_MODEL_NAME = 'neuralmind/bert-base-portuguese-cased'
MAX_LEN = 128
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
bert_model = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

Criando função para gerar embeddings dos textos processados. Primeiro, os textos são tokenizados, truncados ou preenchidos para atingir o comprimento máximo (max_len), depois são convertidos em tensores PyTorch. Esses tensores, incluindo inpuit_ids e attention_masks, são organizados em um DataLoader para processamento em lotes com o modelo BERT. No loop, o modelo gera a saída, da qual é extraída a embedding correspondente ao token [CLS]. Esses embeddings são acumulados, convertidos em arrays NumPy, concatenados e retornados como um único array que representa todas as entradas.

In [24]:
def generate_embeddings(texts, tokenizer, model, max_len, batch_size):
    encoded = tokenizer(
        texts,
        add_special_tokens=True,
        max_length=max_len,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )
    dataset = TensorDataset(encoded['input_ids'], encoded['attention_mask'])
    dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=2)

    embeddings = []
    with torch.no_grad():
        for input_ids, attention_mask in dataloader:
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            cls_embedding = outputs.last_hidden_state[:, 0, :]
            embeddings.append(cls_embedding.cpu().numpy())
    return np.vstack(embeddings)

Obtendo os embeddings:

In [25]:
embeddings_file = "embeddings.npy"

if os.path.exists(embeddings_file):
    embeddings = np.load(embeddings_file)
else:
    texts = df['text'].tolist()
    batch_size = 32
    embeddings = generate_embeddings(texts, tokenizer, bert_model, MAX_LEN, batch_size)
    np.save("embeddings.npy", embeddings)

# **Modelo de Machine Learning:**

In [26]:
labels = df['label'].tolist()
X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.2, random_state=42)

classifier = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=500, random_state=42)
classifier.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=500, random_state=42)

# **Protocolo de experimentos e validação**

In [27]:
y_pred = classifier.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['falso', 'verdadeiro']))

              precision    recall  f1-score   support

       falso       0.96      0.96      0.96       718
  verdadeiro       0.96      0.96      0.96       722

    accuracy                           0.96      1440
   macro avg       0.96      0.96      0.96      1440
weighted avg       0.96      0.96      0.96      1440



Abaixo, criamos uma função para facilitar o processo de testes do modelo com dados novos. Definimos que caso seja fornecido uma frase de entrada completamente fora do escopo do que o modelo conhece ou caso ele não tenha muita certeza, ele deve retornar que não sabe se a notícia é verdade ou mentira, para que não contribua acidentamente para propagação de desinformação.

In [30]:
CONFIDENCE_THRESHOLD = 0.7

def predict_with_threshold(text, tokenizer, model, classifier, max_len, threshold):
    encoded = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_len,
        truncation=True,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = encoded['input_ids'].to(model.device)
    attention_mask = encoded['attention_mask'].to(model.device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        cls_embedding = outputs.last_hidden_state[:, 0, :]

    cls_embedding = cls_embedding.cpu().numpy()
    probas = classifier.predict_proba(cls_embedding)[0]

    max_proba = np.max(probas)
    if max_proba < threshold:
        return "Não sei", max_proba

    return ("Verdadeiro" if np.argmax(probas) == 1 else "Falso"), max_proba

Testando com frases novas

In [29]:
test_sentences = [
    "EUA lançou míssil nuclear.", #resultado esperado = mentira
    "Brasil declarou guerra contra algum país.", #resultado esperado = mentira
    "Guitarrista de banda famosa atira em espectador durante show de rock.", #resultado esperado = mentira
    "Partido diz que apoia a investigação com a ampla apuração dos eventuais crimes cometidos", #resultado esperado = verdade
    "Eduardo Cunha disse em depoimento à Justiça Federal que não recebeu dinheiro da empresa JBS", #resultado esperado = verdade
    "O ministro do STF liberou Andrea Neves da prisão domiciliar" #resultado esperado = verdade
]

for sentence in test_sentences:
    result, confidence = predict_with_threshold(
        sentence, tokenizer, bert_model, classifier, MAX_LEN, CONFIDENCE_THRESHOLD
    )
    print(f"Frase: {sentence}")
    print(f"Resposta: {result}, Probabilidade: {confidence:.2f}\n")

Frase: EUA lançou míssil nuclear.
Resposta: Falso, Probabilidade: 1.00

Frase: Brasil declarou guerra contra algum país.
Resposta: Falso, Probabilidade: 1.00

Frase: Guitarrista de banda famosa atira em espectador durante show de rock.
Resposta: Falso, Probabilidade: 1.00

Frase: Partido diz que apoia a investigação com a ampla apuração dos eventuais crimes cometidos
Resposta: Verdadeiro, Probabilidade: 1.00

Frase: Eduardo Cunha disse em depoimento à Justiça Federal que não recebeu dinheiro da empresa JBS
Resposta: Verdadeiro, Probabilidade: 1.00

Frase: O ministro do STF liberou Andrea Neves da prisão domiciliar
Resposta: Verdadeiro, Probabilidade: 0.90



# **Discussão dos resultados e trabalhos futuros**

Após o treinamento do modelo, efetuamos testes fornecendo frases relacionadas a notícias verdadeiras e notícias falsas que ele possuí conhecimento, e os resultados apresentaram acuracidade alta.

Como a base que utilizamos foi atualizada pela última vez quatro anos atrás, para uma continuação desse projeto seria interessante adicionar noticías referentes à esses anos faltantes, principalmente notícias mais recentes. Essa base deveria ser atualizada constantemente e o modelo retreinado, para que ele sempre possa responder com a maior acuracidade possível para o contexo atual. Como é mais difícil encontrar bases robustas em português, talvez fosse interessante nós mesmo fazermos esse processo de obtenção atráves de portais de notícia confiáveis e portais conhecidos pela divulgação de fake news.